### This notebook preprocess the Amazon Review Dataset for sentiment analysis model training

In [ ]:
# Import necessary libraries
import pandas as pd

In [2]:
df = pd.read_csv(r"D:\ML_Projects\Amazon-Review-Sentiment-Analysis-System\Data\Reviews.csv")

In [3]:
df.head()

,ProfileName,Summary,Text,Score
0,delmartian,Good Quality Dog Food,I have bought several of the Vitality canned d...,5
1,dll pa,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,1
2,"Natalia Corres ""Natalia Corres""","""Delight"" says it all",This is a confection that has been around a fe...,4
3,Karl,Cough Medicine,If you are looking for the secret ingredient i...,2
4,"Michael D. Bigham ""M. Wassir""",Great taffy,Great taffy at a great price. There was a wid...,5


In [6]:
len(df["Text"])

568430